## Cache the output of all the samples

So the next part is the generator of the examples - which should be the easy part!

Lets look at `create_api_calls_summary_doc`

So this creates the directory:

output_directory_name = "documenting_pi_api"
output_file_name = "API_SUMMARY.md"
file_number_equivalent = 1

with open(os.path.join(output_directory_name, output_file_name), "w") as f:

But this was for creating the "document" in process kinda thing?

Isn't what we need actually OutputHandlerV3?

class OutputHandlerV3:
    @staticmethod
    def save_output_to_json(output, method_name, query_type, *args):
        output_directory = "extracted_data"
        os.makedirs(output_directory, exist_ok=True)

        # arg_string = ', '.join(repr(arg) for arg in args).replace("'", "").replace("/", "_")
        query_name = f"index_dot_{method_name}({arg_string})_{query_type}"
        
        file_number = 1
        while True:
            output_file = f"{file_number:03d}_{query_name}.json"
            existing_files = os.listdir(output_directory)
    
            file_exists = False
            for file in existing_files:
                if file.startswith(f"{file_number:03d}_"):
                    file_exists = True
                    break

            if not file_exists:
                break

            file_number += 1

        with open(os.path.join(output_directory, output_file), "w") as f:
            json.dump(output, f, indent=4)

        print("\nOUTPUT CREATED\n")

Actually the problem is I have DELETED HOW THIS IS USED!!!

So Rewind has it (starred and saved):

def create_dir_of_output_examples (api_calls):
   file directory = 'api_outputs_all_examples'
    for idx, api_call in valid_api_calls:
        method = api_call ['method']
        query_type = api_call ['query_type']
        query = api call['query']
        main_data_field = api_call ['main_data_field']
        method_to_call = getattr (api_instance.index, method)
        result = method_to_call(query)
        OutputHandlerV3.save_output_to_json(result, method, query_type, query)
print(result)

So lets delete create_api_calls_summary_doc for now.  We can search for it easily enough with CMD+SHIFT+F.

#### What does it need?

From the list... all I think it needs is a api_call['method'] 

Oh, and we need the valid query as well

In [ ]:
api_calls = ['search', 'podcastByFeedUrl', 'podcastByFeedId', 'podcastByItunesId', 'episodesByFeedUrl', 'episodesByFeedId', 'episodesByItunesId', 'episodeById', 'episodesByPerson', 'recentEpisodes', 'trendingPodcasts']
query = 'TBC'

for api_call in api_calls:
        
        method = api_call ['method']
        method_to_call = getattr(api_instance.index, method)
        
        valid_query = [query]

        payload = method_to_call(query)

In [ ]:
# So this is what we have for valid_queries:

valid_queries = {'search_term': 'python', 'person': 'Guido van Rossum', 'feed_id': 227573, 'itunes_id': 1325018583, 'feed_url': 'https://podnews.net/rss', 'episode_id': 15120462484}

# So don't we just need to change the list of methods into a dictionary associating each method with the required query value?

api_calls = ['search', 'podcastByFeedUrl', 'podcastByFeedId', 'podcastByItunesId', 'episodesByFeedUrl', 'episodesByFeedId', 'episodesByItunesId', 'episodeById', 'episodesByPerson', 'recentEpisodes', 'trendingPodcasts']

# so it'd be: {method : query_type}

api_calls = {
    'search': 'search_term', 
    'podcastByFeedUrl': 'feed_url', 
    'podcastByFeedId': 'feed_id', 
    'podcastByItunesId':'itunes_id', 
    'episodesByFeedUrl': 'feed_url', 
    'episodesByFeedId': 'feed_id', 
    'episodesByItunesId': 'itunes_id', 
    'episodeById': 'episode_id', 
    'episodesByPerson': 'person', 
    'recentEpisodes': 10, 
    'trendingPodcasts': 10}

Lets do some experiments with combining these queries correctly.

In [ ]:
method_query_calls = {
    'search': 'search_term', 
    'podcastByFeedUrl': 'feed_url', 
    'podcastByFeedId': 'feed_id', 
    'podcastByItunesId':'itunes_id', 
    'episodesByFeedUrl': 'feed_url', 
    'episodesByFeedId': 'feed_id', 
    'episodesByItunesId': 'itunes_id', 
    'episodeById': 'episode_id', 
    'episodesByPerson': 'person', 
    'recentEpisodes': 'max', 
    'trendingPodcasts': 'max'
    }

valid_queries = {
    'search_term': 'python',
    'person': 'Guido van Rossum', 
    'feed_id': 227573, 
    'itunes_id': 1325018583, 
    'feed_url': 'https://podnews.net/rss', 
    'episode_id': 15120462484,
    'max': 10,
    }


for method, query in method_query_calls.items():
    query = valid_queries[query]
    print(method, query)

### Building up the function

So I think we are pretty close to having this.

Here is what we have:

In [ ]:
from dev_tools.api_dev_tools import PodcastIndexConfig, PodcastIndexAPI

method_query_calls = {
    'search': 'search_term', 
    'podcastByFeedUrl': 'feed_url', 
    'podcastByFeedId': 'feed_id', 
    'podcastByItunesId':'itunes_id', 
    'episodesByFeedUrl': 'feed_url', 
    'episodesByFeedId': 'feed_id', 
    'episodesByItunesId': 'itunes_id', 
    'episodeById': 'episode_id', 
    'episodesByPerson': 'person', 
    'recentEpisodes': 'max', 
    'trendingPodcasts': 'max'
    }

valid_queries = {
    'search_term': 'python',
    'person': 'Guido van Rossum', 
    'feed_id': 227573, 
    'itunes_id': 1325018583, 
    'feed_url': 'https://podnews.net/rss', 
    'episode_id': 15120462484,
    'max': 10,
    }

config_instance = PodcastIndexConfig()
api_instance = PodcastIndexAPI(config_instance.config)

for method, query in method_query_calls.items():
    query = valid_queries[query]
    method_to_call = getattr(api_instance.index, method)
    # payload = method_to_call(query)
    print(method_to_call)

Next thing is the json part - which is just OutputHandlerV3?

Before that, lets do a test, to see if it's actually working.

(This could be a unit test for sure but lets just get this moving here.)

Lets return the data in the returned calls.

So that is going to need a list of the meat-fields...

In [ ]:
main_data_field_per_api_method = [
    {'search': 'feeds'}, 
    {'podcastByFeedUrl': 'feed'}, 
    {'podcastByFeedId': 'feed'}, 
    {'podcastByItunesId': 'feed'}, 
    {'episodesByFeedUrl': 'items'}, 
    {'episodesByFeedId': 'items'}, 
    {'episodesByItunesId': 'items'}, 
    {'episodeById': 'episode'}, 
    {'episodesByPerson': 'items'}, 
    {'recentEpisodes': 'items'}, 
    {'trendingPodcasts': 'feeds'}
]

print(main_data_field_per_api_method)

In [ ]:
from dev_tools.api_dev_tools import PodcastIndexConfig, PodcastIndexAPI

method_query_calls = {
    'search': 'search_term', 
    'podcastByFeedUrl': 'feed_url', 
    'podcastByFeedId': 'feed_id', 
    'podcastByItunesId':'itunes_id', 
    'episodesByFeedUrl': 'feed_url', 
    'episodesByFeedId': 'feed_id', 
    'episodesByItunesId': 'itunes_id', 
    'episodeById': 'episode_id', 
    'episodesByPerson': 'person', 
    'recentEpisodes': 'max', 
    'trendingPodcasts': 'max'
    }


So it seems like we could make things more general purpose by combining them, so I think that's a good idea.  Chat GPT suggests

In [ ]:
api_methods = {
    'search': {'query_type': 'search_term', 'main_data_field': 'feeds'},
    'podcastByFeedUrl': {'query_type': 'feed_url', 'main_data_field': 'feed'},
    'podcastByFeedId': {'query_type': 'feed_id', 'main_data_field': 'feed'},
    'podcastByItunesId': {'query_type': 'itunes_id', 'main_data_field': 'feed'},
    'episodesByFeedUrl': {'query_type': 'feed_url', 'main_data_field': 'items'},
    'episodesByFeedId': {'query_type': 'feed_id', 'main_data_field': 'items'},
    'episodesByItunesId': {'query_type': 'itunes_id', 'main_data_field': 'items'},
    'episodeById': {'query_type': 'episode_id', 'main_data_field': 'episode'},
    'episodesByPerson': {'query_type': 'person', 'main_data_field': 'items'},
    'recentEpisodes': {'query_type': 'max', 'main_data_field': 'items'},
    'trendingPodcasts': {'query_type': 'max', 'main_data_field': 'feeds'}
}

# api_method = 'search'
# query_type = api_methods[api_method]['query_type']
# main_data_field = api_methods[api_method]['main_data_field']

valid_queries = {
    'search_term': 'python', 'person': 'Guido van Rossum', 'feed_id': 227573, 'itunes_id': 1325018583, 'feed_url': 'https://podnews.net/rss', 'episode_id': 15120462484, 'max': 10,
    }

for method, query_info in api_methods.items():
    print(method)
    print(query_info['query_type'])
    print(query_info['main_data_field'])
    
    query = valid_queries[query_info['query_type']]
    print(query)
    
    main_data_field = query_info['main_data_field']
    print(main_data_field)
    
    # method_to_call = getattr(api_instance.index, method)
    # payload = method_to_call(query)
    # print(method_to_call)

Lets try and return basic info from each payload:

In [ ]:
### needs to run in a separate file

from dev_tools.api_dev_tools import PodcastIndexConfig, PodcastIndexAPI

config_instance = PodcastIndexConfig()
api_instance = PodcastIndexAPI(config_instance.config)

api_methods = {
    'search': {'query_type': 'search_term', 'main_data_field': 'feeds'},
    'podcastByFeedUrl': {'query_type': 'feed_url', 'main_data_field': 'feed'},
    'podcastByFeedId': {'query_type': 'feed_id', 'main_data_field': 'feed'},
    'podcastByItunesId': {'query_type': 'itunes_id', 'main_data_field': 'feed'},
    'episodesByFeedUrl': {'query_type': 'feed_url', 'main_data_field': 'items'},
    'episodesByFeedId': {'query_type': 'feed_id', 'main_data_field': 'items'},
    'episodesByItunesId': {'query_type': 'itunes_id', 'main_data_field': 'items'},
    'episodeById': {'query_type': 'episode_id', 'main_data_field': 'episode'},
    'episodesByPerson': {'query_type': 'person', 'main_data_field': 'items'},
    'recentEpisodes': {'query_type': 'max', 'main_data_field': 'items'},
    'trendingPodcasts': {'query_type': 'max', 'main_data_field': 'feeds'}
}

# api_method = 'search'
# query_type = api_methods[api_method]['query_type']
# main_data_field = api_methods[api_method]['main_data_field']

valid_queries = {
    'search_term': 'python', 'person': 'Guido van Rossum', 'feed_id': 227573, 'itunes_id': 1325018583, 'feed_url': 'https://podnews.net/rss', 'episode_id': 15120462484, 'max': 10,
    }

for method, query_info in api_methods.items():
    print(f"The method is: {method}")
    print(f"The query type is: {query_info['query_type']}")
        
    query = valid_queries[query_info['query_type']]
    print(f"The query is: {query}")
    
    main_data_field = query_info['main_data_field']
    print(f"The main data is in: {main_data_field}")
    
    method_to_call = getattr(api_instance.index, method)
    payload = method_to_call(query)
    
    main_data = payload[main_data_field]
    if type(main_data) == list:
        sample_dict = main_data[0]
    else:
        sample_dict = main_data

    print(list(sample_dict.values()))
    print()

So that works completely up to the episodebyid feed - which from my samples returns a dictionary but seems to have been treated as a list. 

Odd.

But anyway, it's as much of a proof of concept as I need.  Lets bring over outputhandler.

In [15]:
import os
import json
from dev_tools.api_dev_tools import PodcastIndexConfig, PodcastIndexAPI

api_methods = {
    'search': {'query_type': 'search_term', 'main_data_field': 'feeds'},
    'podcastByFeedUrl': {'query_type': 'feed_url', 'main_data_field': 'feed'},
    'podcastByFeedId': {'query_type': 'feed_id', 'main_data_field': 'feed'},
    'podcastByItunesId': {'query_type': 'itunes_id', 'main_data_field': 'feed'},
    'episodesByFeedUrl': {'query_type': 'feed_url', 'main_data_field': 'items'},
    'episodesByFeedId': {'query_type': 'feed_id', 'main_data_field': 'items'},
    'episodesByItunesId': {'query_type': 'itunes_id', 'main_data_field': 'items'},
    'episodeById': {'query_type': 'episode_id', 'main_data_field': 'episode'},
    'episodesByPerson': {'query_type': 'person', 'main_data_field': 'items'},
    'recentEpisodes': {'query_type': 'max', 'main_data_field': 'items'},
    'trendingPodcasts': {'query_type': 'max', 'main_data_field': 'feeds'}
}

valid_queries = {
    'search_term': 'python', 'person': 'Guido van Rossum', 'feed_id': 227573, 'itunes_id': 1325018583, 'feed_url': 'https://podnews.net/rss', 'episode_id': 15120462484, 'max': 10,
    }

def save_output_to_json(payload, method):
        output_directory = "pi_output_cache/sample_api_response"
        os.makedirs(output_directory, exist_ok=True)
       
        file_number = 1
        while True:
            output_file = f"{file_number:03d}_{method}.json"
            existing_files = os.listdir(output_directory)
    
            file_exists = False
            for file in existing_files:
                if file.startswith(f"{file_number:03d}_"):
                    file_exists = True
                    break

            if not file_exists:
                break

            file_number += 1

        with open(os.path.join(output_directory, output_file), "w") as f:
            json.dump(payload, f, indent=4)

def download_sample_set(api_methods, valid_queries):
    for method, query_info in api_methods.items():
        query = valid_queries[query_info['query_type']]
        method_to_call = getattr(api_instance.index, method)
        payload = method_to_call(query)
        print(f"For the method {method} -- the query type used: {query_info['query_type']} -- the query: {query}")
        save_output_to_json(payload, method) 

config_instance = PodcastIndexConfig()
api_instance = PodcastIndexAPI(config_instance.config)

download_sample_set(api_methods, valid_queries)


For the method search -- the query type used: search_term -- the query: python
For the method podcastByFeedUrl -- the query type used: feed_url -- the query: https://podnews.net/rss
For the method podcastByFeedId -- the query type used: feed_id -- the query: 227573
For the method podcastByItunesId -- the query type used: itunes_id -- the query: 1325018583
For the method episodesByFeedUrl -- the query type used: feed_url -- the query: https://podnews.net/rss
For the method episodesByFeedId -- the query type used: feed_id -- the query: 227573
For the method episodesByItunesId -- the query type used: itunes_id -- the query: 1325018583
For the method episodeById -- the query type used: episode_id -- the query: 15120462484
For the method episodesByPerson -- the query type used: person -- the query: Guido van Rossum
For the method recentEpisodes -- the query type used: max -- the query: 10
For the method trendingPodcasts -- the query type used: max -- the query: 10


So it worked! Episode id is invalid is the only issue - (which caused the other problem as there was no allowance in any of that for invalid replies).

So I think we are now able to pull all this together. 

